<a href="https://colab.research.google.com/github/YatindraRai002/Quiz/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report




In [ ]:
df = pd.read_csv('/content/adult.csv', index_col=0, na_values=["?"])

In [ ]:
df.size

348852

In [ ]:
df.head(10)

,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
age,,,,,,,,,,,,,,
90,NaN,77053,HS-grad,9,Widowed,NaN,Not-in-family,White,Female,0,4356,40.0,United-States,<=50K
82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18.0,United-States,<=50K
66,NaN,186061,Some-college,10,Widowed,NaN,Unmarried,Black,Female,0,4356,40.0,United-States,<=50K
54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40.0,United-States,<=50K
41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40.0,United-States,<=50K
34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45.0,United-States,<=50K
38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40.0,United-States,<=50K
74,State-gov,88638,Doctorate,16,Never-married,Prof-specialty,Other-relative,White,Female,0,3683,20.0,United-States,>50K
68,Federal-gov,422013,HS-grad,9,Divorced,Prof-specialty,Not-in-family,White,Female,0,3683,40.0,United-States,<=50K


In [ ]:
df.tail(10)

,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
age,,,,,,,,,,,,,,
39,Private,262841,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50.0,United-States,>50K
55,Private,37438,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,>50K
22,Private,170800,Assoc-voc,11,Never-married,Other-service,Own-child,White,Female,0,0,12.0,United-States,<=50K
44,Private,152150,Assoc-acdm,12,Separated,Exec-managerial,Not-in-family,White,Male,0,0,40.0,United-States,<=50K
44,Private,159580,12th,8,Divorced,Transport-moving,Not-in-family,White,Female,0,0,40.0,United-States,<=50K
61,Private,477209,7th-8th,4,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,54.0,United-States,<=50K
32,Private,70985,Assoc-voc,11,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40.0,United-States,<=50K
35,Private,241998,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45.0,United-States,>50K
28,Private,249541,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States,<=50K


In [ ]:
from sklearn.impute import SimpleImputer

X = df.drop('income', axis=1)
Y = df['income'].fillna('').apply(lambda x: 1 if x.strip()=='>50K' else 0)

num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler())]), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
pipe_lr = Pipeline([
    ('pre', preprocessor),
    ('clf', LogisticRegression(penalty='l2', solver='saga', max_iter=10000))
])
param_lr = {'clf__C': [0.01, 0.1, 1, 10]}
grid_lr = GridSearchCV(pipe_lr, param_lr, cv=cv, scoring='accuracy', n_jobs=-1)
grid_lr.fit(X, Y)
print("Best LR params:", grid_lr.best_params_)
print(grid_lr.best_score_)


Best LR params: {'clf__C': 0.1}
0.8492657539647643


In [ ]:
pipe_svm = Pipeline([
    ('pre', preprocessor),
    ('clf', SVC(kernel='rbf'))
])
param_svm = {'clf__C': [0.1, 1, 10], 'clf__gamma': ['scale', 'auto']}
grid_svm = GridSearchCV(pipe_svm, param_svm, cv=cv, scoring='accuracy', n_jobs=-1)
grid_svm.fit(X, Y)
print("Best SVM params:", grid_svm.best_params_)
print(grid_svm.best_score_)


Best SVM params: {'clf__C': 1, 'clf__gamma': 'scale'}
0.8510852090032154


In [ ]:
pipe_rf = Pipeline([
    ('pre', preprocessor),
    ('clf', RandomForestClassifier(random_state=42))
])
param_rf = {'clf__n_estimators': [100, 200],
            'clf__max_depth': [10, 20, None],
            'clf__min_samples_leaf': [1, 5, 10]}
grid_rf = GridSearchCV(pipe_rf, param_rf, cv=cv, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X, Y)
print("Best RF params:", grid_rf.best_params_)
print(grid_rf.best_score_)


Best RF params: {'clf__max_depth': 20, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 100}
0.8622588424437299


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=42)
best_models = {
    'LR': grid_lr.best_estimator_,
    'SVM': grid_svm.best_estimator_,
    'RF': grid_rf.best_estimator_
}
for name, model in best_models.items():
    model.fit(X_train, Y_train)
    preds = model.predict(X_test)
    print(f"\n{name} Test Accuracy:", accuracy_score(Y_test, preds))
    print(classification_report(Y_test, preds))



LR Test Accuracy: 0.8561093247588425
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      3703
           1       0.77      0.63      0.69      1273

    accuracy                           0.86      4976
   macro avg       0.82      0.78      0.80      4976
weighted avg       0.85      0.86      0.85      4976


SVM Test Accuracy: 0.8557073954983923
              precision    recall  f1-score   support

           0       0.87      0.94      0.91      3703
           1       0.78      0.60      0.68      1273

    accuracy                           0.86      4976
   macro avg       0.83      0.77      0.79      4976
weighted avg       0.85      0.86      0.85      4976


RF Test Accuracy: 0.8591237942122186
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      3703
           1       0.80      0.60      0.69      1273

    accuracy                           0.86      4976
   macro avg 